In [ ]:
import torch
import numpy as np
import wandb

from lumen_irmas import get_loader, ASTPretrained, parse_config, Learner
from lumen_irmas.modeling.learner import KDLearner
from lumen_irmas.modeling.models import StudentAST, Ensemble

In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
def main(config):
    
    
    wandb.init(config=config, anonymous="allow")
    
    train_dl = get_loader(config, subset="train")
    valid_dl = get_loader(config, subset="valid")
    
    student = StudentAST(11, hidden_size=384, num_attention_heads=6)
    
    weight_list = ["bpmsync_2", "no_sync1"]
    weights = [f"../weights/{weight}.pth" for weight in weight_list]
    base_model = ASTPretrained(n_classes=11)
    teachers = Ensemble(base_model, weights)
    thresholds = []

    learn = KDLearner(train_dl, valid_dl, student, teachers, thresholds, config)

    learn.fit()
    
    wandb.finish()

In [ ]:
if __name__=="__main__":

    CONFIG_PATH = "../configs/KD_config.yaml"
    config = parse_config(CONFIG_PATH)

    main(config)

wandb: Currently logged in as: k-pintaric. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.bias', 'classifier.layernorm.weight', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.bias', 'classifier.layernorm.weight', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a mo

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/22166 [00:00<?, ?it/s]

/nfs/home/8548f215c44990b1/lumen-irmas/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
